In [4]:
import os, getpass
import numpy as np
import pandas as pd

from configparser import ConfigParser
from cax.qsub import submit_job
import time

In [2]:
class Submit(object):
    '''
        1) arg to loop over 
        2) maximum number of nodes to use at once
        3) maximum number of jobs to submit
    '''
    def name(self):
        return self.__class__.__name__

    def execute(self, *args, **kwargs):
        eval('self.{name}(*args, **kwargs)'.format(name = self.name().lower()))

    def submit(self, loop_over=[], max_node_in_use=10, max_job=3):
        _start = 2
        self.max_node_in_use = max_node_in_use
        self.loop_over = loop_over

        index = _start
        while (index < len(self.loop_over) and index < max_job):
            if (self.working_job() < self.max_node_in_use):
                self._submit_single(loop_index=index,
                                    loop_item=self.loop_over[index])

                time.sleep(0.1)
                index += 1

    # check my jobs
    def working_job(self):
        cmd='squeue --user={user} | wc -l'.format(user = 'zhut')
        jobNum=int(os.popen(cmd).read())
        return  jobNum -1

    def _submit_single(self, loop_index, loop_item):
        JOBNAME='paxwf{:03}'.format(loop_index)
        RUNNAME=str(loop_item)

        cmd = """#!/bin/bash
#SBATCH --job-name={job_name}
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem-per-cpu=6000
#SBATCH --output={log}/{run_name}.log
#SBATCH --error={err}/{run_name}.log
#SBATCH --account=pi-lgrandi
#SBATCH --partition={partition}
#SBATCH --qos={qos}

export PATH="$/cvmfs/xenon.opensciencegrid.org/releases/anaconda/2.4/bin:$PATH"
cd {cwd}

source activate {conda_env}
python GrowMTs.py {run_name}
"""

        log_directory = os.path.join(os.getcwd(), 'log')
        if not os.path.isdir(log_directory):
            os.mkdir(log_directory)

        y = cmd.format(job_name = 'submi{:03}'.format(loop_index),
                       run_name = loop_item,
                       log = log_directory,
                       err = log_directory,
                       tmp = os.path.join(os.getcwd(), 'tmp'),
                       cwd = os.getcwd(),
                       partition = 'dali',
                       qos = 'dali',
                       conda_env = 'pax_head',
                      )
        #print(y)
        submit_job(y)


In [3]:
p = Submit()
loop_over = np.arange(0, 20)
p.execute(loop_over=loop_over, max_num_submit=50, nmax=50)